<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/ELMo_on_IEMOCAP_for_ERC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


**This is for ELMo**

In [2]:
!pip install tensorflow-hub

In [3]:
import tensorflow_hub as hub
import tensorflow as tf

elmo = hub.Module("https://tfhub.dev/google/elmo/2", trainable=True)

In [4]:
def elmo_vectors(x):
  embeddings = elmo(x, signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

**This is for Preprocessing and model**

In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
import pickle
import pandas as pd
import numpy as np
import keras
import itertools
from sklearn.model_selection import train_test_split
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional, Input
from keras.optimizers import Adam
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical  

from keras.layers import Layer
import keras.backend as K

Using TensorFlow backend.


In [7]:
with open('/content/gdrive/My Drive/iemocap/train/sentences.pkl', 'rb') as f:
    data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/train/labels.pkl', 'rb') as f:
    labels = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/sentences.pkl', 'rb') as f:
    test_data = pickle.load(f)
with open('/content/gdrive/My Drive/iemocap/test/labels.pkl', 'rb') as f:
    test_labels = pickle.load(f)

In [8]:
def preprocessing(data,labels):
  processed_data=[]
  processed_label=[]
  for i in range(0,len(data)):
    for j in range(0,len(data[i])):
      intermediate_data=[]
      intermediate_label=[]
      for k in range(0,len(data[i][j])):
        text=data[i][j][k]
        if text != '<eos>'and text!='<pad>':
          intermediate_data.append(text)
      processed_data.append(intermediate_data)
  for i in labels:
    for j in i:
      processed_label.append(j)
  return processed_data,processed_label

In [9]:
processed_data,processed_label = preprocessing(data,labels)
test_processed_data,test_processed_label = preprocessing(test_data,test_labels)

In [10]:
for i in range(0,len(processed_data)):
  processed_data[i]= ' '.join(processed_data[i])

In [11]:
for i in range(0,len(test_processed_data)):
  test_processed_data[i]=' '.join(test_processed_data[i])

In [31]:
def create_batches(data):
  count=0
  temp=[]
  batches=[]
  for i in data:
    count=count+1
    temp.append(i)
    if count%100==0:
      count=0
      batches.append(temp)
      temp=[]
  return batches

In [35]:
list_train = create_batches(processed_data)
list_test = create_batches(test_processed_data)

In [ ]:
elmo_train = elmo_vectors(processed_data)
elmo_test = elmo_vectors(test_processed_data)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
# save elmo_train_new
pickle_out = open("elmo_train_03032019.pickle","wb")
pickle.dump(elmo_train_new, pickle_out)
pickle_out.close()

# save elmo_test_new
pickle_out = open("elmo_test_03032019.pickle","wb")
pickle.dump(elmo_test_new, pickle_out)
pickle_out.close()